In [2]:
import random
import math

# invert a dictionary of lists (assuming no duplicates)
def invert_dictlist(d):
    return dict( (v,k) for k in d for v in d[k] )

# invert a dictionary of lists (with duplicates)
def invert_dictlist_dup(d):
    values = set(a for b in d.values() for a in b)
    reverse_d = dict((new_key, [key for key,value in d.items() if new_key in value]) for new_key in values)
    return reverse_d


In [120]:
# assign students in groups to k submissions.
def peer_assignment(groups,k,debug=False):
    """Given no cover, first generate a cover with the first few submissions"""
    """Then, generate the rest of the assignments"""
    
    submissions = groups.keys()
    
    # lookup for which submissions to exclude from a particular student.
    exclude = invert_dictlist(groups)
    students = exclude.keys()
    studentList = list(students);

 
    # load = ceil(number of students * k / number of submissions)
    # this is how many copies of random submission lists we need.
    load = int(math.ceil((len(students) * k) / len(submissions)))
    
    # Repeat the submissions so that they each occur load times.
    repeatedSubmissions = [x for x in submissions for i in range(load-1)]
    
    # initialize empty assignment and cover
    assignments = {s : [] for s in students}
    cover = {s : [] for s in students}
    
    # cover with the first few submissions. Every time a submission is used, remove it.
    # This way, each submission is repeated at most load # of times.
    for s in students:
        for currentSubmission in repeatedSubmissions:
            if currentSubmission not in exclude[s]:
                assignments[s].append(currentSubmission);
                repeatedSubmissions.remove(currentSubmission);
                cover[s].append(currentSubmission);
                break;
    
    assignments = peer_assignment_with_cover(groups,k,cover);
    
    if (assignments == -1) and debug:
        print("Couldn't generate good cover!");
    
    done = True;
    for s in students:
        if (len(assignments[s]) != k):
            done = False;
            
    if not(done) and debug:
        print("Not a good assignment!")
        return -1;
    
    return assignments;
    
def peer_assignment_with_cover_submissions(groups,k,coverSubmissions,debug=False):
    """Given a list of submissions, generate a cover using those submissions"""
    """Then, generate the rest of the assignments"""
    
    submissions = groups.keys()
    
    # lookup for which submissions to exclude from a particular student.
    exclude = invert_dictlist(groups)
    students = exclude.keys()
    studentList = list(students);

    # load = ceil(number of students * k / number of submissions)
    # this is how many copies of random submission lists we need.
    load = int(math.ceil((len(students) * k) / len(submissions)))
    
    repeatedCoverSubmissions = [x for x in coverSubmissions for i in range(load-1)]
    
    # initialize empty cover
    cover = {s : [] for s in students}
    
    # cover with the first few of coverSubmissions
    for s in students:
        for currentSubmission in repeatedCoverSubmissions:
            if currentSubmission not in exclude[s]:
                repeatedCoverSubmissions.remove(currentSubmission);
                cover[s].append(currentSubmission);
                break;
    
    for s in students:
        if len(assignments[s]) == 0:
            if (debug):
                print("Error: Submissions cannot cover students!")
            return -1;
        
    assignments = peer_assignment_with_cover(groups,k,cover);
    
    if (assignments == -1) and debug:
        print("Couldn't generate good cover!");
        return -1;
    
     
    done = True;
    for s in students:
        if (len(assignments[s]) != k):
            done = False;
            
    if not(done) and debug:
        print("Not a good assignment!")
        return -1;
    
    return assignments;
    
def peer_assignment_with_cover(groups,k,cover,debug=False):
    """Given an entire cover of (student,submission) pairs, generate the rest of the assignments"""
    
    
    for i in range(10000):
        
        submissions = groups.keys()
        submissionsList = list(submissions);
        
        # lookup for which submissions to exclude from a particular student.
        exclude = invert_dictlist(groups)
        students = exclude.keys()
        studentList = list(students);
    
     
        # load = ceil(number of students * k / number of submissions)
        # this is how many copies of random submission lists we need.
        load = int(math.ceil((len(students) * k) / len(submissions)))
    
        # Repeat the submissions so that they each occur load times.
        extras = len(students)*k - len(submissions)*(load - 1);
        repeatedSubmissions = [x for x in submissions for i in range(load-1)]
        more = submissionsList[:extras]
        repeatedSubmissions = repeatedSubmissions + more
        
    
        # Assign cover, and remove covered submissions from submissions list
        assignments = cover;
        for s in students:
            for currentSubmission in assignments[s]:
                repeatedSubmissions.remove(currentSubmission);
        
        permutedSubmissions = random.sample(repeatedSubmissions,len(repeatedSubmissions));
        repeatedStudents = [x for x in studentList for i in range(k-1)]
        permutedStudents = random.sample(repeatedStudents,len(repeatedStudents));
    
        for s in permutedStudents:
            for currentSubmission in permutedSubmissions:
                assignments[s].append(currentSubmission);
                if not(check_assignment(groups,assignments)):
                    assignments[s].remove(currentSubmission);
                else:
                    permutedSubmissions.remove(currentSubmission);
                    break;
        
        done = True;
        
        for s in students:
            if (len(assignments[s]) != k):
                done = False;
        
        if done:
            break;
    
    done = True;
    for s in students:
        if (len(assignments[s]) != k):
            done = False;
    
    #print(load)
    #print(len(submissions))
    #print(len(students))
    #print(extras)
    
    if not(done) and debug:
        print("Bad Cover!")
        return -1;
      
    # print the cover
    # here one can also output to file, etc.
    #print("Cover: ");
    #print(invert_dictlist_dup(cover));
    
    return assignments
    

# make sure nobody is assigned own assignment or duplicates
def check_assignment(groups,assignments):
    # maps students to their own submission
    
    exclude = invert_dictlist(groups)

    passed = True
    
    for s in assignments.keys():
        if exclude[s] in assignments[s]:
            #print("Student " + s + " assigned to own submission\n")
            passed = False
        if len(assignments[s]) != len(set(assignments[s])):
            #print("Student " + s + " assigned duplicate submissions\n")
            passed = False
            
    return passed

In [121]:
# generate some group data
groups = { sub : [sub + x for x in ['1','2','3']] for sub in [ chr(ord('a') + z) for z in range(10)]}

# get assignments
assignments = peer_assignment(groups,3)

# print assignments
invert_dictlist_dup(assignments)

{'a': ['j3', 'i3', 'e1', 'g1', 'c1', 'j1', 'd1', 'h3', 'i2'],
 'b': ['i1', 'h2', 'a2', 'f1', 'e1', 'e3', 'h1', 'g1', 'd1'],
 'c': ['b2', 'a1', 'f2', 'b1', 'i1', 'h2', 'f3', 'h1', 'i2'],
 'd': ['a3', 'g2', 'c2', 'f2', 'i1', 'a2', 'g1', 'j1', 'h3'],
 'e': ['g2', 'd2', 'b2', 'b3', 'f1', 'g3', 'i3', 'c3', 'h3'],
 'f': ['e2', 'j2', 'a1', 'h2', 'i3', 'h1', 'c1', 'd1', 'c3'],
 'g': ['j2', 'a3', 'b1', 'd3', 'f3', 'c1', 'j1', 'h3', 'i2'],
 'h': ['e2', 'j2', 'a3', 'b1', 'j3', 'd3', 'f3', 'g3', 'e3'],
 'i': ['c2', 'd2', 'b3', 'j3', 'd3', 'a2', 'f1', 'g3', 'e3'],
 'j': ['g2', 'c2', 'd2', 'b2', 'b3', 'a1', 'f2', 'e1', 'c3']}

In [122]:
peer_assignment(groups,3,True)

{'a1': ['j', 'b', 'e'],
 'a2': ['b', 'i', 'e'],
 'a3': ['h', 'g', 'd'],
 'b1': ['h', 'a', 'i'],
 'b2': ['f', 'j', 'd'],
 'b3': ['j', 'a', 'g'],
 'c1': ['a', 'e', 'i'],
 'c2': ['j', 'i', 'f'],
 'c3': ['j', 'f', 'g'],
 'd1': ['a', 'f', 'g'],
 'd2': ['j', 'i', 'e'],
 'd3': ['g', 'h', 'c'],
 'e1': ['b', 'a', 'f'],
 'e2': ['h', 'i', 'd'],
 'e3': ['b', 'f', 'd'],
 'f1': ['b', 'e', 'g'],
 'f2': ['j', 'c', 'g'],
 'f3': ['h', 'c', 'j'],
 'g1': ['b', 'e', 'f'],
 'g2': ['j', 'i', 'c'],
 'g3': ['h', 'd', 'f'],
 'h1': ['b', 'e', 'i'],
 'h2': ['b', 'g', 'i'],
 'h3': ['a', 'e', 'd'],
 'i1': ['b', 'e', 'g'],
 'i2': ['a', 'c', 'd'],
 'i3': ['a', 'c', 'h'],
 'j1': ['a', 'f', 'c'],
 'j2': ['h', 'd', 'c'],
 'j3': ['h', 'd', 'c']}

In [118]:
# generate groups

L = [chr(ord('a') + z) for z in range(26)];
bigL = [x + y for x in L for y in L]
bigGroups = {sub : [sub + x for x in ['1','2','3']] for sub in bigL[:90]};
bigGroups['aa'].remove('aa1');
bigGroups['ab'].remove('ab1');
bigGroups['ac'].remove('ac1');
bigGroups['ad'].remove('ad1');
bigGroups['ae'].remove('ae1');
bigGroups['af'].remove('af1');
bigGroups['ag'].remove('ag1');
bigGroups['ah'].remove('ah1');
bigGroups['ai'].remove('ai1');
bigGroups['aj'].remove('aj1');
bigGroups['ak'].remove('ak1');
bigGroups['al'].remove('al1');
bigGroups['am'].remove('am1');
bigGroups['an'].remove('an1');
bigGroups['ao'].remove('ao1');
bigGroups['ap'].remove('ap1');
bigGroups['aq'].remove('aq1');
bigGroups['ar'].remove('ar1');
bigGroups['as'].remove('as1');
bigGroups['at'].remove('at1');
bigGroups['au'].remove('au1');
bigGroups['av'].remove('av1');
bigGroups['aw'].remove('aw1');
bigGroups['ax'].remove('ax1');
bigGroups['ay'].remove('ay1');
bigGroups['az'].remove('az1');


# generate assignments
debug = True;
assignments = peer_assignment(bigGroups,3,debug)

# assignments
invert_dictlist_dup(assignments)

9
90
244
12


{'aa': ['dg2', 'bo3', 'be1', 'cq2', 'di1', 'bz2', 'cu1', 'ch3'],
 'ab': ['de2', 'cu2', 'ak2', 'at3', 'bm3', 'cb3', 'cc1', 'dj2'],
 'ac': ['ak3', 'ag2', 'as2', 'dh3', 'df3', 'bd1', 'cm2', 'cu1'],
 'ad': ['ax3', 'dc3', 'ao3', 'af2', 'af3', 'bs1', 'cm1', 'ce2'],
 'ae': ['dk1', 'bf3', 'cl1', 'bw1', 'cb1', 'cd3', 'df1', 'az3'],
 'af': ['av3', 'bk1', 'aj2', 'dj3', 'am3', 'ct2', 'br3', 'bs1'],
 'ag': ['cf2', 'cg2', 'ck2', 'bk3', 'cz1', 'cy1', 'cq3', 'br1'],
 'ah': ['cu2', 'ck2', 'bo2', 'au3', 'ag2', 'ce3', 'bo1', 'cd1', 'di3'],
 'ai': ['cd2', 'bp3', 'cx1', 'cw1', 'bc2', 'ay3', 'aq2', 'ch3'],
 'aj': ['dk1', 'ac2', 'ab3', 'ci3', 'ag3', 'az2', 'be2', 'cb3'],
 'ak': ['dl2', 'bf2', 'bq3', 'ai3', 'bj1', 'cn2', 'cs3', 'bb1'],
 'al': ['cs2', 'av3', 'ay2', 'bk1', 'bo1', 'cj3', 'as3', 'cv2'],
 'am': ['bk3', 'be3', 'cp3', 'df3', 'cl2', 'bg3', 'cd3', 'ca3'],
 'an': ['cp2', 'bv3', 'bq1', 'cu3', 'ba2', 'dh2', 'ba1', 'ay3'],
 'ao': ['bf3', 'aa3', 'ah3', 'ac3', 'dd3', 'cw2', 'bl1', 'br3'],
 'ap': ['dj1', 'cr

In [119]:
peer_assignment(bigGroups,3)

9
90
244
12


{'aa2': ['cc', 'ai', 'cs'],
 'aa3': ['ao', 'ab', 'bk'],
 'ab2': ['bg', 'bu', 'ci'],
 'ab3': ['co', 'dh', 'bi'],
 'ac2': ['df', 'bl', 'cj'],
 'ac3': ['ao', 'de', 'au'],
 'ad2': ['bb', 'cb', 'cn'],
 'ad3': ['co', 'cb', 'by'],
 'ae2': ['cf', 'dj', 'cj'],
 'ae3': ['bf', 'cb', 'ay'],
 'af2': ['ad', 'bv', 'ak'],
 'af3': ['ad', 'aj', 'cn'],
 'ag2': ['ah', 'bp', 'bc'],
 'ag3': ['cc', 'ab', 'ac'],
 'ah2': ['dl', 'bn', 'dg'],
 'ah3': ['ao', 'ae', 'bs'],
 'ai2': ['bd', 'cx', 'cu'],
 'ai3': ['aw', 'ba', 'bl'],
 'aj2': ['dc', 'cp', 'cj'],
 'aj3': ['bv', 'cm', 'cr'],
 'ak2': ['df', 'dh', 'az'],
 'ak3': ['bv', 'cu', 'dd'],
 'al2': ['bv', 'az', 'ch'],
 'al3': ['ca', 'az', 'au'],
 'am2': ['bo', 'bh', 'de'],
 'am3': ['aw', 'bu', 'aa'],
 'an2': ['bo', 'bg', 'af'],
 'an3': ['bz', 'bm', 'cz'],
 'ao2': ['dl', 'al', 'ae'],
 'ao3': ['ad', 'cz', 'da'],
 'ap2': ['ax', 'ch', 'av'],
 'ap3': ['cf', 'ba', 'be'],
 'aq2': ['bd', 'bl', 'bt'],
 'aq3': ['cc', 'bu', 'ak'],
 'ar2': ['bo', 'by', 'bq'],
 'ar3': ['aq', 'cl',